# Snake

We're going to create the classic game of snake, which we've played at Grandma and Grandad's house many times.
We'll start with the basic game, but if you want to extend it to do anything else you can do.

## Getting Started

* Switch to the terminal window from PyCharm
* Install the pygame library by typing `pip install pygame`

TODO: Multiplayer Snake with Guns!

In [8]:
# import the pygame module - make all the functionality in there available
# This original code isn't going to end, so you'll need to manually stop it by hitting the red square at the top of the page

import pygame

# start the game module
pygame.init()

# Create a display window size 400 pixels by 300 pixels
display_window = pygame.display.set_mode((400,300))

pygame.display.update()
pygame.display.set_caption('SNACKEN IS HUNGRY')

game_over=False
while not game_over:
    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            game_over=True
pygame.quit()
quit()

Next we need to make some things called "Sprites" a sprite represents a thing on screen
The snake itself is going to be a bit trickier, but fortunately other people out there have made snake games...

If you look for snake sprites images, you can find sprite sheets.  I found one at https://rembound.com/files/creating-a-snake-game-tutorial-with-html5/snake-graphics.png and saved it locally to images/snake.png.  The sprites in here are size 16x16

The code below is a baseline to work from.  The snake will grow indefinitely and eventually go off the top of the screen
There are placeholders marked with TODO to add additional logic in


In [2]:
import pygame
from enum import IntEnum

# width and height are measured in tile size (eg 16 x 16 pixels)
SCREEN_WIDTH = 40
SCREEN_HEIGHT = 30
TILE_SIZE = 16

# How many times we repeat the render loop per second
# we render each time the snake moves so actually want a really slow render
# 2 will render twice a second
FPS = 2

# Some magic I grabbed from https://ehmatthes.github.io/pcc_2e/beyond_pcc/pygame_sprite_sheets/
# but didn't spend any time trying to understand
class SpriteSheet:
    def __init__(self, filename):
        self.sheet = pygame.image.load(filename).convert()

    def image_at(self, rectangle, colorkey = None):
        rect = pygame.Rect(rectangle)
        image = pygame.Surface(rect.size).convert()
        image.blit(self.sheet, (0, 0), rect)
        if colorkey is not None:
            if colorkey == -1:
                colorkey = image.get_at((0,0))
            image.set_colorkey(colorkey, pygame.RLEACCEL)
        return image

    def images_at(self, rects, colorkey = None):
        return [self.image_at(rect, colorkey) for rect in rects]

    def load_strip(self, rect, image_count, colorkey = None):
        tuples = [(rect[0]+rect[2]*x, rect[1], rect[2], rect[3])
                for x in range(image_count)]
        return self.images_at(tuples, colorkey)

class SnakeDirection(IntEnum):
    Up = 0
    Right = 1
    Down = 2
    Left = 3

class SnakeImage(IntEnum):
    HeadUp = 0
    HeadRight = 1
    HeadDown = 2
    HeadLeft = 3
    TailUp = 4
    TailRight = 5
    TailDown = 6
    TailLeft = 7
    BodyTurnUpRight = 8
    BodyTurnDownRight = 9
    BodyTurnDownLeft = 10
    BodyTurnUpLeft = 11
    BodyVertical = 12
    BodyHorizontal = 13
    Rabbit = 14

    def get_image(self, sprite_sheet):
        return sprite_sheet.image_at(((int(self) % 4) * TILE_SIZE, (int(self / 4)) * TILE_SIZE, TILE_SIZE, TILE_SIZE))

class Sprite(pygame.sprite.Sprite):
    def __init__(self, snake_image, rect_on_screen):
        super(Sprite, self).__init__()
        self.image = snake_image
        self.rect = rect_on_screen

class SnakeGame:
    def __init__(self):

        pygame.init()
        self.display_window = pygame.display.set_mode((SCREEN_WIDTH * TILE_SIZE, SCREEN_HEIGHT * TILE_SIZE))
        self.clock = pygame.time.Clock()

        self.sprite_sheet = SpriteSheet("images/Snake.png")
        self.snake_sprites = pygame.sprite.Group()
        self.rabbit_sprites = pygame.sprite.Group()

        self.add_rabbit()

        pygame.display.update()
        pygame.display.set_caption('SNACKEN IS HUNGRY.  SNACKEN HAS NOT BEEN FED')

        # initialise game here
        self.current_direction = SnakeDirection.Up
        self.current_location = (SCREEN_WIDTH * TILE_SIZE / 2, SCREEN_HEIGHT * TILE_SIZE / 2)

        # make initial snake
        self.snake_head = Sprite(SnakeImage.HeadUp.get_image(self.sprite_sheet), (self.current_location[0], self.current_location[1], TILE_SIZE, TILE_SIZE))
        self.snake_tail = Sprite(SnakeImage.TailUp.get_image(self.sprite_sheet), (self.current_location[0], self.current_location[1] + TILE_SIZE, TILE_SIZE, TILE_SIZE))
        self.snake_sprites.add(self.snake_head)
        self.snake_sprites.add(self.snake_tail)
        self.snake_size = 2

        # TODO: add targets to get

    def has_snake_crashed(self):
        return False

    def add_rabbit(self):
        # Currently hardcoded position
        rabbit = Sprite(SnakeImage.Rabbit.get_image(self.sprite_sheet), (300, 64, 16, 16))
        self.rabbit_sprites.add(rabbit)

        # TODO: add this to a random location on screen
        # preferably not on top of the snake's tail!

    def render_sprites(self, sprites):
        sprites.update()
        for sprite in sprites:
            self.display_window.blit(sprite.image, sprite.rect)

    def grow_snake_up(self, last_direction):
        self.snake_sprites.remove(self.snake_head)
        if last_direction == SnakeDirection.Left:
            self.snake_head.image = SnakeImage.BodyTurnUpLeft.get_image(self.sprite_sheet)
        elif last_direction == SnakeDirection.Right:
            self.snake_head.image = SnakeImage.BodyTurnUpRight.get_image(self.sprite_sheet)
        else:
            self.snake_head.image = SnakeImage.BodyVertical.get_image(self.sprite_sheet)
        self.current_location = (self.current_location[0], self.current_location[1] - TILE_SIZE)
        self.snake_sprites.add(self.snake_head)
        self.snake_head = Sprite(SnakeImage.HeadUp.get_image(self.sprite_sheet), (self.current_location[0], self.current_location[1], TILE_SIZE, TILE_SIZE))
        self.snake_sprites.add(self.snake_head)

    def grow_snake_right(self, last_direction):
        self.current_location = (self.current_location[0] + TILE_SIZE, self.current_location[1])
        # TODO: add extra code here

    def grow_snake_down(self, last_direction):
        self.current_location[1] = (self.current_location[0], self.current_location[1] + TILE_SIZE)
        # TODO: add extra code here

    def grow_snake_left(self, last_direction):
        self.current_location[0] = (self.current_location[0] - TILE_SIZE, self.current_location[1])
        # TODO: add extra code here

    def remove_snake_tail(self):
        # TODO: remove the tail of the snake and update the next one along to be the new tail
        print("remove_snake_tail not implemented yet")

    def main(self):

        is_game_over = False
        while not is_game_over:
            last_direction = self.current_direction

            for gameEvent in pygame.event.get():
                if gameEvent.type == pygame.QUIT:
                    is_game_over=True
                if gameEvent.type == pygame.KEYDOWN:
                    if gameEvent.key == pygame.K_UP:
                        self.current_direction = SnakeDirection.Up
                    elif gameEvent.key == pygame.K_RIGHT:
                        self.current_direction = SnakeDirection.Right
                    elif gameEvent.key == pygame.K_DOWN:
                        self.current_direction = SnakeDirection.Down
                    elif gameEvent.key == pygame.K_LEFT:
                        self.current_direction = SnakeDirection.Left

            # grow the snake in the current current_direction
            if self.current_direction == SnakeDirection.Up:
                self.grow_snake_up(last_direction)
            elif self.current_direction == SnakeDirection.Right:
                self.grow_snake_right(last_direction)
            elif self.current_direction == SnakeDirection.Down:
                self.grow_snake_down(last_direction)
            elif self.current_direction == SnakeDirection.Left:
                self.grow_snake_left(last_direction)

            if self.snake_sprites.__sizeof__() > self.snake_size:
                self.remove_snake_tail()

            # TODO: add some logic to eat things and grow the snake

            is_game_over = is_game_over or self.has_snake_crashed()

            # display stuff into a virtual whiteboard
            background = (0, 200, 0) # red, green, blue values (255 max)
            self.display_window.fill(background)
            self.render_sprites(self.snake_sprites)
            # flip the whiteboard so it's visible
            pygame.display.flip()

            self.clock.tick(FPS)

        pygame.quit()
        quit()

game = SnakeGame()
game.main()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
not implemented yet
